In [6]:
# !pip install dotenv
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 10.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 

In [7]:
import os
import sys
from dotenv import load_dotenv, find_dotenv

class Utils:
    def __init__(self):
        pass

    def create_dlai_index_name(self, index_name):
        """Create a unique index name based on the Gemini API key (or part of it)."""
        gemini_key = ''
        if self.is_colab():
            from google.colab import userdata
            gemini_key = userdata.get("GEMINI_API_KEY")
        else:
            gemini_key = os.getenv("GEMINI_API_KEY")

        # Use a hashed or masked portion of the key if needed
        suffix = gemini_key[-12:].lower().replace("_", "-") if gemini_key else "no-key"
        return f'{index_name}-{suffix}'

    def is_colab(self):
        """Detect if the code is running in Google Colab."""
        return 'google.colab' in sys.modules

    def get_gemini_api_key(self):
        """Load Gemini API key from environment or .env file."""
        _ = load_dotenv(find_dotenv())
        return os.getenv("GEMINI_API_KEY")

    def get_pinecone_api_key(self):
        """Load Pinecone API key from environment or .env file."""
        _ = load_dotenv(find_dotenv())
        return os.getenv("PINECONE_API_KEY")

In [8]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
gemini_api_key = user_secrets.get_secret("GOOGLE_API_KEY")
pinecone_api_key = user_secrets.get_secret("PINECONE_API_KEY")

In [9]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

import os
import time
import torch

In [10]:
from tqdm.auto import tqdm

In [11]:
dataset = load_dataset('quora', split='train[240000:290000]')

README.md: 0.00B [00:00, ?B/s]

quora.py: 0.00B [00:00, ?B/s]

The repository for quora contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/quora.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/404290 [00:00<?, ? examples/s]

In [12]:
dataset[:5]

{'questions': [{'id': [207550, 351729],
   'text': ['What is the truth of life?', "What's the evil truth of life?"]},
  {'id': [33183, 351730],
   'text': ['Which is the best smartphone under 20K in India?',
    'Which is the best smartphone with in 20k in India?']},
  {'id': [351731, 351732],
   'text': ['Steps taken by Canadian government to improve literacy rate?',
    'Can I send homemade herbal hair oil from India to US via postal or private courier services?']},
  {'id': [37799, 94186],
   'text': ['What is a good way to lose 30 pounds in 2 months?',
    'What can I do to lose 30 pounds in 2 months?']},
  {'id': [351733, 351734],
   'text': ['Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?',
    'How do you graph x + 2y = -2?']}],
 'is_duplicate': [False, True, False, True, False]}

In [13]:
questions = []
for record in dataset['questions']:
    questions.extend(record['text'])
question = list(set(questions))
print('\n'.join(questions[:10]))
print('-' * 50)
print(f'Number of questions: {len(questions)}')

What is the truth of life?
What's the evil truth of life?
Which is the best smartphone under 20K in India?
Which is the best smartphone with in 20k in India?
Steps taken by Canadian government to improve literacy rate?
Can I send homemade herbal hair oil from India to US via postal or private courier services?
What is a good way to lose 30 pounds in 2 months?
What can I do to lose 30 pounds in 2 months?
Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?
How do you graph x + 2y = -2?
--------------------------------------------------
Number of questions: 100000


In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('Sorry no cuda.')
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
query = 'which city is the most populated in the world?'
xq = model.encode(query)
xq.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(384,)

In [16]:
utils = Utils()
PINECONE_API_KEY = utils.get_pinecone_api_key()

In [19]:
pinecone = Pinecone(api_key=pinecone_api_key)
INDEX_NAME = utils.create_dlai_index_name('dl-ai')
print(INDEX_NAME)

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    pinecone.delete_index(INDEX_NAME)
print(INDEX_NAME)
pinecone.create_index(name=INDEX_NAME, 
    dimension=model.get_sentence_embedding_dimension(), 
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1'))

index = pinecone.Index(INDEX_NAME)
print(index)

dl-ai-no-key
dl-ai-no-key


In [20]:
batch_size=200
vector_limit=10000

questions = question[:vector_limit]

import json

for i in tqdm(range(0, len(questions), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(questions))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadatas = [{'text': text} for text in questions[i:i_end]]
    # create embeddings
    xc = model.encode(questions[i:i_end])
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

  0%|          | 0/50 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [21]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 9600}},
 'total_vector_count': 9600,
 'vector_type': 'dense'}

In [22]:
# small helper function so we can repeat queries later
def run_query(query):
  embedding = model.encode(query).tolist()
  results = index.query(top_k=10, vector=embedding, include_metadata=True, include_values=False)
  for result in results['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

In [23]:
run_query('which city has the highest population in the world?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.67: Which city in the world is the most beautiful to live in?
0.64: Which city has the most museums per capita?
0.61: Which city deserves to be the Capital of the World?
0.61: What is the most modern city in China?
0.58: What country has the most beautiful people?
0.56: What percentage of the world's population lives in developing countries?
0.55: Which country is the largest democracy in the world?
0.55: What is the recent population of India?
0.54: What are the largest slums in the world?
0.52: What is the biggest arena in the world?


In [24]:
query = 'how do i make chocolate cake?'
run_query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.45: How does one make rice pudding in a rice cooker?
0.45: Which is the weirdest birthday cake ever?
0.51: Is cake considered bread?
0.43: How do I make green tea?
0.45: Why is banana bread considered a bread and not a cake?
0.49: What is the difference between chocolate and truffles?
0.52: How do you bake air-dry clay?
0.61: What is a cake mix?
0.56: What are the best date cake recipes?
0.43: Is it safe for dogs to eat chocolate? If not, what can you do if your dog accidentally eats some?
